In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import numpy as np
import torch

import gzip
import os
import pickle

import importlib
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import os
os.chdir('../../price_model/')


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


In [3]:
# Sentence Transformer from Huggingface library
from sentence_transformers import SentenceTransformer 


/Users/willtong/Library/Caches/pypoetry/virtualenvs/wine-libraries-PLl6gzSz-py3.12/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Load data set

In [4]:
df = pd.read_json("../data/deduped_mod_winemag-data-130k-v2.json")
df.shape

(111593, 15)

## Here I augment the rest of the wine information into the the text.  No need to worry about grammatically correctness or punctuation.

In [5]:


info_features = ['title',  'designation', 'variety', 'region-1',
       'region-2', 'province', 'country', 'winery']
df.fillna(" ", inplace = True)
df["merged_info_text"] = df[['description', "merged_info_text"]].apply(lambda info_features: ' '.join(list(info_features)), axis = 1)
df["len"] = df["merged_info_text"].apply(lambda text: len(text.split(" ")))


df.drop(info_features, axis = 1, inplace = True)

In [6]:
df

,points,description,taster-name,taster-twitter-handle,price,merged_info_text,len
1,87,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",Roger Voss,@vossroger,15,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016. This ripe fruity, wine smooth still structured. Firm tannins filled juicy red berry fruits freshened acidity. It's already drinkable, although certainly better 2016. Quinta Douro (Douro) Portugal Red 2011 Avidagos Portuguese",70
2,87,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",Paul Gregutt,@paulgwine,14,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented. Tart snappy, flavors lime flesh rind dominate. Some green pineapple pokes through, crisp acidity underscoring flavors. The wine stainless-steel fermented. Pinot (Willamette Valley 2013 US Rainstorm Gris Oregon Willamette Valley)",58
3,87,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",Alexander Peartree,,13,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish. Pineapple rind, lemon pith orange blossom start aromas. The palate bit opulent, notes honey-drizzled guava mango giving way slightly astringent, semidry finish. Shore) St. Shore 2013 Riesling US Lake Harvest Late Michigan Reserve (Lake Julian",68
4,87,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",Paul Gregutt,@paulgwine,65,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew. Much like regular bottling 2012, comes across rather rough tannic, rustic, earthy, herbal characteristics. Nonetheless, think pleasantly unfussy country wine, good companion hearty winter stew. Pinot 2012 (Willamette Block Vintner's Valley Sweet Cheeks Willamette US Noir Oregon Child Reserve Wild Valley)",82
5,87,"Blackberry and raspberry aromas show a typical Navarran whiff of green herbs and, in this case, horseradish. In the mouth, this is fairly full bodied, with tomatoey acidity. Spicy, herbal flavors complement dark plum fruit, while the finish is fresh but grabby.",Michael Schachner,@wineschach,15,"Blackberry and raspberry aromas show a typical Navarran whiff of green herbs and, in this case, horseradish. In the mouth, this is fairly full bodied, with tomatoey acidity. Spicy, herbal flavors complement dark plum fruit, while the finish is fresh but grabby. Blackberry raspberry aromas show typical Navarran whiff green herbs and, case, horseradish. In mouth, fairly full bodied, tomatoey acidity. Spicy, herbal flavors complement dark plum fruit, finish fresh grabby. (Navarra) 2011 Tandem Northern Spain Ars Tempranillo-Merlot Vitro In Navarra",81
...,...,...,...,...,...,...,...
129966,90,"Notes of honeysuckle and cantaloupe sweeten this deliciously feather-light spätlese. It's intensely juicy, quenching the palate with strea

In [7]:
# The removes the ceiling for memory usage for application on my laptop.  
# It would run out of money if I don't remove it.
# Exercise extreme caution.   
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '-0.0'

## Maximum length of the text is 225 words, this is very close to the upper limits of most models of 256 words.  That's why I choose the model with the highest word limit of 512: 'msmarco-distilbert-dot-v5'


In [10]:
df["len"].max()

225

In [11]:
# This is the sentence transformer that can take the longest text (512 words).   
# Our description can be as long as 225 words (near 256 words which is the limit of next best model. 
# Wheir embeddings will not be perfect, still well over half their words are captured.
# It is the best we can do.
model_names = ['msmarco-distilbert-dot-v5']
trust_remote_code = False

# I used the following to generate embeddings with multiple transformers and compare.  
# Here I have only one.
for model_name in model_names:
    model = SentenceTransformer(model_name, trust_remote_code = trust_remote_code)
    df["desc_embed"] = df["merged_info_text"].apply(lambda text: model.encode(text, convert_to_tensor=True))
    desc_embeds = model.encode(df["merged_info_text"].values, convert_to_tensor=False)
    df_embeds = pd.DataFrame(columns = ['embed_'+ str(num).zfill(3) for num in range(desc_embeds.shape[1])],
                             data = desc_embeds,
                             index = df.index
                            )
    with gzip.open("../data/embedded_desc_nostopwords"+model_name.split("/")[-1]+".pckl", "wb") as f:
        pickle.dump(df_embeds, f)